# Chapter 3: Unit Testing - Turning Specs into Checks

## What is Unit Testing?

Unit testing is a common software testing technique to check individual
code components (e.g., functions, methods, classes, or modules) in isolation from
the rest of the program.

The main idea is to run such isolated components with a variety of inputs and check the
outputs against expected results.

For unit testing to be achievable and effective, the code design must facilitate
easy isolation of components and their dependencies. In line with the design
principles we discussed earlier, below are some key practices to follow:

 - **Small, cohesive functions:** A function should do one thing clearly (single responsibility). Recall how we applied this principle when reimplementing the heat equation solver.

 - **Explicit interfaces:** Functions and methods should have clear and explicit interfaces (e.g., well-defined input parameters and return values). This makes it easier to understand how to use them and to test them in isolation.

 - **Avoid side effects:** Functions should avoid modifying global state or relying on external state. This makes it easier to reason about their behavior and to test them in isolation.

 - **Use dependency injection:** Instead of hardcoding dependencies, pass them as parameters. This makes it easier to replace them with mocks or stubs during testing.  

 ## Benefits of Unit Testing

 - **Catches bugs early:** Unit tests can help identify bugs and issues in the code at an early stage.

 - **Facilitates code changes:** With a comprehensive suite of unit tests, developers can make changes to the code with confidence, knowing that any regressions will be caught by the tests.

 - **Improves code design:** Writing unit tests encourages developers to think about the design and structure of their code, leading to better-organized and more maintainable code.

 - **Documentation:** Unit tests serve as a form of documentation, providing examples of how to use the code and what its expected behavior is.



## Unit Testing in Action

Take the `flux` function from the previous chapter as an example. (We have split 
the assertion checks into separate lines for clarity.)

In [66]:
def flux(F: list, u: list, kappa: float, dx: float) -> None:
    """Update face fluxes F based on cell field u."""

    assert dx > 0, "Invalid grid spacing"
    assert len(F) == len(u) + 1, "Inconsistent face flux array length"

    for i in range(1, len(u)):
        F[i] = -kappa * (u[i] - u[i-1]) / dx

Unit testing this function is as simple as calling it with some test inputs and checking the outputs. Here's how you might write a unit test for the `flux` function using `pytest`:

In [67]:
from numpy import isclose

def test_flux():
    """Constant field leads to zero flux"""
    u = [100.0, 100.0, 100.0]
    F = [0.0] * (len(u) + 1)

    flux(F, u, kappa=0.1, dx=1.0)
    assert all(isclose(F[1:-1], 0)), f"Expected all zeros, got {F[1:-1]}"

test_flux()

That's it! We unit tested the `flux` function. And the test passed successfully.
How do we know? The assertion did not raise an error.

But we only did so for a specific case, where we confirmed that a constant field 
and zero boundary fluxes lead to zero flux. We'll add more tests to cover different scenarios,
but first let's introduce the `pytest` framework.

## The `pytest` Library

While manual testing is useful, it can be time-consuming and error-prone. Automated testing with a framework like `pytest` allows us to quickly and easily run our tests, for instance, when added into a continuous integration (CI) pipeline.

While there are other commonly used frameworks such as `unittest`, we prefer
`pytest` for its simplicity and powerful features. It's also worth noting that `pytest`
is not just for unit testing: it's a general purpose testing framework that can be used
for a wide range of testing needs in an automated fashion.

Before we begin describing `pytest`, it's worth mentioning that it is a command-line tool. 
Therefore, to be able to run it through this notebook interface, we will follow the following
workflow:

1. Using the `%%writefile` magic command, we will save our test codes to Python files.
2. We will then run the tests using the command line command `pytest`. Recall, in Jupyter notebooks, we can run such shell commands by prefixing them with `!`.

First, create a source file named `heat1d.py` that contains the heat-equation solver we
developed in Chapter 2:

In [68]:
%%writefile heat1d.py

from dataclasses import dataclass
from numpy import isclose

# Data abstractions

@dataclass
class Mesh:
    """Uniform 1-D mesh."""
    dx: float
    N: int  # number of cells
    def cell_centered_arr(self): return [0.0] * self.N
    def face_centered_arr(self): return [0.0] * (self.N + 1)

@dataclass
class BC:
    """Neumann flux boundary conditions on the diffusive flux."""
    qL: float
    qR: float

# Behavior abstractions

def apply_bc(F: list, qL: float, qR: float) -> None:
    """Apply Neumann BCs to face fluxes F."""
    assert len(F) > 1
    F[0], F[-1] = qL, qR

def flux(F: list, u: list, kappa: float, dx: float) -> None:
    """Update face fluxes F based on cell field u."""

    assert dx > 0, "Invalid grid spacing"
    assert len(F) == len(u) + 1, "Inconsistent face flux array length"

    for i in range(1, len(u)):
        F[i] = -kappa * (u[i] - u[i-1]) / dx

def divergence_is_conservative(F, dudt, dx):
    """Check if divergence is conservative."""
    total_divergence = sum(dudt) * dx
    boundary_flux = F[0] - F[-1]
    return isclose(total_divergence, boundary_flux)

def divergence(dudt: list, F: list, dx: float) -> None:
    """Update cell tendencies dudt based on face fluxes F."""
    assert len(F) == len(dudt) + 1
    for i in range(len(dudt)):
        dudt[i] = (F[i] - F[i+1]) / dx
    assert divergence_is_conservative(F, dudt, dx)

def step(u: list, F: list, dudt: list, kappa: float, dt: float, mesh: Mesh, bc: BC) -> list:
    """Advance cell field u by one time step using explicit Euler method."""
    assert dt > 0

    apply_bc(F, bc.qL, bc.qR)
    flux(F, u, kappa, mesh.dx)
    divergence(dudt, F, mesh.dx)

    for i in range(mesh.N):
        u[i] += dt * dudt[i]

def Euler_FTCS_stability(dt: float, dx: float, kappa: float) -> bool:
    """Check stability condition for explicit Euler (FTCS) scheme."""
    r = kappa * dt / dx**2
    return r <= 0.5

def heat_is_conserved(u_sum_before: float, u_sum_after: float, dt: float, bc: BC, mesh: Mesh) -> bool:
    """Check if heat is conserved."""
    expected_change = dt * (bc.qL - bc.qR)
    actual_change = u_sum_after - u_sum_before
    return isclose(actual_change, expected_change)

def solve(u0: list, kappa: float, dt: float, nt: int, dx: float, bc: BC) -> list:
    """Orchestrate nt steps over cell field u."""

    assert nt > 0
    assert Euler_FTCS_stability(dt, dx, kappa), "Stability condition not met"

    mesh = Mesh(dx, N=len(u0))
    u = u0.copy()
    F = mesh.face_centered_arr()
    dudt = mesh.cell_centered_arr()

    for _ in range(nt):
        u_sum_before = sum(u) * mesh.dx
        step(u, F, dudt, kappa, dt, mesh, bc)
        u_sum_after = sum(u) * mesh.dx
        assert heat_is_conserved(u_sum_before, u_sum_after, dt, bc, mesh), "Heat not conserved"

    return u


Overwriting heat1d.py


In the remainder of this section, we will import this `heat1d.py` module to define
unit tests for its functions.


## Key Features of `pytest`

### Assertions

pytest uses plain Python assert statements, no special API, to decide whether a test passes.
It also rewrites assertions to show the full expression and the actual values on failure, 
which makes errors easy to diagnose. When you expect an error, use the `pytest.raises` context manager to assert that a specific exception is raised.

Be careful with floating-point comparisons: exact equality is brittle. In tests, 
prefer `pytest.approx` for tolerant comparisons. Using `numpy.isclose` inside your
library code is fine (as we did in the source code: `heat1d.py`). In tests, `approx`
tends to produce clearer failure messages.

Because we embedded pre- and postconditions as asserts in our solver, those checks run 
automatically whenever tests execute the code. Just remember they only fire for the specific
inputs your tests provide. On top of those contracts, you’ll typically add test-side assertions
that express the behavior you want to verify.

Pytest discovers tests by looking for functions whose names start with `test_`. Each such
function is executed as an independent test: if an assertion fails, pytest reports the failure
(with values); if an unexpected exception occurs, it reports that too.

Let’s write a minimal test for the `div` function. In practice, source code and test are 
often located in separate files. But for brevity, we’ll keep them together and save to
`test_div.py`:

In [69]:
%%writefile test_div.py

def div(x, y):
    assert y != 0           # P    (precondition)
    res = x / y             # code (implementation)
    assert res * y == x     # Q    (postcondition)
    return res

def test_division():
    div(7, 25)

Overwriting test_div.py


To run this test, we can simply run the `pytest test_div.py` command:

In [70]:
!pytest test_div.py

============================= test session starts ==============================
platform darwin -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0
rootdir: /Users/altuntas/r3sw/notebooks
plugins: hypothesis-6.136.9, anyio-4.10.0
collected 1 item                                                               

test_div.py F                                                            [100%]

=================================== FAILURES ===================================
________________________________ test_division _________________________________

    def test_division():
>       div(7, 25)

test_div.py:9: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 

x = 7, y = 25

    def div(x, y):
        assert y != 0           # P    (precondition)
        res = x / y             # code (implementation)
>       assert res * y == x     # Q    (postcondition)
        ^^^^^^^^^^^^^^^^^^^
E       assert (0.28 * 25) == 7

test_div.py:5: AssertionError
=================

Notice the test fails because the assertion is violated due to floating point precision of division operation. We may address this test failure in several ways. First, we can
mark the test as expected to fail using the `@pytest.mark.xfail` decorator.

In [71]:
%%writefile test_div.py

import pytest

def div(x, y):
    assert y != 0           # P    (precondition)
    res = x / y             # code (implementation)
    assert res * y == x     # Q    (postcondition)
    return res

@pytest.mark.xfail
def test_division():
    res = div(7, 25)

Overwriting test_div.py


In [72]:
!pytest test_div.py

============================= test session starts ==============================
platform darwin -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0
rootdir: /Users/altuntas/r3sw/notebooks
plugins: hypothesis-6.136.9, anyio-4.10.0
collected 1 item                                                               

test_div.py x                                                            [100%]

============================== 1 xfailed in 0.23s ==============================


Alternatively, we can use the `raises` context manager:

In [73]:
%%writefile test_div.py

from pytest import raises

def div(x, y):
    assert y != 0           # P    (precondition)
    res = x / y             # code (implementation)
    assert res * y == x     # Q    (postcondition)
    return res

def test_division():
    with raises(AssertionError):
        res = div(7, 25)

Overwriting test_div.py


In [74]:
!pytest test_div.py

============================= test session starts ==============================
platform darwin -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0
rootdir: /Users/altuntas/r3sw/notebooks
plugins: hypothesis-6.136.9, anyio-4.10.0
collected 1 item                                                               

test_div.py .                                                            [100%]

============================== 1 passed in 0.24s ===============================


Or, more appropriately for this situation, we can weaken the postcondition by replacing the equality assertion with an approximate equality assertion using the `pytest.approx` function:

In [75]:
%%writefile test_div.py

from pytest import approx

def div(x, y):
    assert y != 0               # P    (precondition)
    res = x / y                 # code (implementation)
    assert res * y == approx(x) # Q    (postcondition)
    return res

def test_division():
    res = div(7, 25)

Overwriting test_div.py


In [76]:
!pytest test_div.py

============================= test session starts ==============================
platform darwin -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0
rootdir: /Users/altuntas/r3sw/notebooks
plugins: hypothesis-6.136.9, anyio-4.10.0
collected 1 item                                                               

test_div.py .                                                            [100%]

============================== 1 passed in 0.25s ===============================


Back to the heat-equation solver: execute the following cell to save this minimal test 
definition as `test_simple.py`

In [77]:
%%writefile test_simple.py

import pytest
from heat1d import flux

def test_flux_simple():
    u = [100, 100, 100]
    kappa = 0.1
    dx = 1.0
    F = [0.0] * (len(u) + 1)
    flux(F, u, kappa, dx)
    assert F[1:-1] == pytest.approx([0.0, 0.0])

Overwriting test_simple.py


To execute this test, we will simply run the `pytest` command in the terminal.

In [78]:
!pytest test_simple.py

============================= test session starts ==============================
platform darwin -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0
rootdir: /Users/altuntas/r3sw/notebooks
plugins: hypothesis-6.136.9, anyio-4.10.0
collected 1 item                                                               

test_simple.py .                                                         [100%]

============================== 1 passed in 0.22s ===============================


The output confirms that the test has passed successfully, i.e., no (unexpected) assertion errors were raised during the test execution.


### Parameterization

You may have realized that running a test with different inputs can be tedious if we have to write separate test functions for each case. To ease this process, you can use the `@pytest.mark.parametrize` decorator to run a test function with different sets of input data.


In [79]:
%%writefile test_flux_via_params.py

import pytest
from pytest import approx
from heat1d import flux

@pytest.mark.parametrize(
    "u,kappa,dx,expected",
    [
        ([100,100,100], 0.1, 1.0, [0.0, 0.0]),
        ([0,10,20],     0.5, 2.0, [-0.5*(10/2), -0.5*(10/2)]),
    ],
    ids=["constant→zero", "linear→const-flux"]
)
def test_flux_param(u, kappa, dx, expected):
    print(f"\nTesting u={u}, kappa={kappa}, dx={dx}")
    F = [0.0]*(len(u)+1)
    flux(F, u, kappa, dx)
    assert F[1:-1] == approx(expected)

Overwriting test_flux_via_params.py


In [80]:
!pytest -s test_flux_via_params.py

============================= test session starts ==============================
platform darwin -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0
rootdir: /Users/altuntas/r3sw/notebooks
plugins: hypothesis-6.136.9, anyio-4.10.0
collected 2 items                                                              

test_flux_via_params.py 
Testing u=[100, 100, 100], kappa=0.1, dx=1.0
.
Testing u=[0, 10, 20], kappa=0.5, dx=2.0
.

============================== 2 passed in 0.23s ===============================


Note: The `-s` flag in the above call is to allow print statements in the test to be displayed,
and so to confirm that all specified inputs via the parameterization mechanism are being tested.
In the absence of this flag, the print statements are suppressed.

### Test Discovery

 `pytest` automatically discovers tests by looking for files that start with `test_` or end with `_test.py`. Within each of these files, it looks for functions that start with `test_` and classes starting with `Test`. All discovered tests are then executed when you run `pytest`.


 Say, you run `pytest` in a directory with the following structure:

```
heat_solver/
    └── heat1d.py
    └── unit_tests/
        ├── test_simple.py
        └── test_flux_via_params.py
```

When you execute `pytest` from the root directory, it will recursively discover and run the tests in all the modules starting with `test_`. Since we have saved two test files so far,
this means that both `test_simple.py` and `test_flux_via_params.py` will be executed.

In [81]:
!pytest

============================= test session starts ==============================
platform darwin -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0
rootdir: /Users/altuntas/r3sw/notebooks
plugins: hypothesis-6.136.9, anyio-4.10.0
collected 14 items                                                             

test_div.py .                                                            [  7%]
test_divergence.py .                                                     [ 14%]
test_flux.py ..                                                          [ 28%]
test_flux_via_params.py ..                                               [ 42%]
test_simple.py .                                                         [ 50%]
test_step_solve.py .......                                               [100%]

============================== 14 passed in 0.26s ==============================



### Fixtures

pytest fixtures give you small, named pieces of test state, e.g., parameters, meshes, boundary conditions, that pytest builds and injects into tests by name. They remove duplicated setup, 
keep tests independent, and make intent explicit. Pytest discovers fixtures in any test file 
and in a shared `conftest.py`, so you can reuse them across modules.

In this chapter we’ll use a few simple fixtures throughout:

 - `dx`, `kappa`: canonical numerical parameters.
 - `mesh3`, `mesh5`: tiny meshes for hand-checkable and slightly larger cases.
 - `insulated`, `linear_bc`: boundary-condition objects.
 - `F3`, `dudt3`: Flux and tendency arrays sized to mesh3.
 - `u_spike`, `u_uniform`: representative initial conditions.

You’ll see these fixtures appear as function arguments in the tests that follow. pytest creates them automatically and passes them in. This keeps each test concise and focused.

In [82]:
%%writefile conftest.py
"""
Shared pytest fixtures for the heat-1D solver. These create small, well-labeled
objects we can reuse across tests without repeating setup code.
"""
import pytest
from heat1d import Mesh, BC

@pytest.fixture
def dx(): return 1.0

@pytest.fixture
def kappa(): return 0.1

@pytest.fixture
def mesh3(dx): return Mesh(dx=dx, N=3)

@pytest.fixture
def mesh5(dx): return Mesh(dx=dx, N=5)

@pytest.fixture
def insulated(): return BC(qL=0.0, qR=0.0)

@pytest.fixture
def linear_bc(): return BC(qL=1.0, qR=1.0)

@pytest.fixture
def F3(mesh3): return mesh3.face_centered_arr()

@pytest.fixture
def dudt3(mesh3): return mesh3.cell_centered_arr()

@pytest.fixture
def u_spike(): return [0.0, 100.0, 0.0]

@pytest.fixture
def u_uniform(mesh5): return [7.5] * mesh5.N

Overwriting conftest.py


## More examples

### Flux tests

Below are a couple of tests for the flux function.

In `test_flux_constant_field_yields_zero_interior`, we show that a constant temperature field
has zero interior gradients, so the interior fluxes F[1:-1] must be zero. Boundary faces are
governed by boundary conditions and are intentionally left untouched by flux.

In `test_flux_spike_has_opposite_signed_fluxes`, the spike profile [0, 100, 0] produces a
positive gradient at the first interior face and a negative gradient at the second, so the 
interior fluxes must have equal magnitude and opposite sign (i.e., [-κ·100/Δx, +κ·100/Δx]). 
Using the `F3` and `u_spike` fixtures keeps the setup clear and state isolated.


In [83]:
%%writefile test_flux.py
from pytest import approx, raises
from heat1d import flux

def test_flux_constant_field_yields_zero_interior():
    """Given a constant field, interior gradients are zero → interior fluxes are zero."""
    u = [100.0, 100.0, 100.0]
    F = [9.9, 0.0, 0.0, 8.8]  # sentinels at boundaries to show they're not changed
    flux(F, u, kappa=0.1, dx=1.0)
    assert F[1:-1] == approx([0.0, 0.0])
    assert F[0] == 9.9 and F[-1] == 8.8  # boundary faces untouched by flux()

def test_flux_spike_has_opposite_signed_fluxes(F3, u_spike):
    """A positive jump then negative jump should produce equal/opposite interior fluxes."""
    kappa, dx = 0.1, 1.0
    F = F3[:]
    flux(F, u_spike, kappa, dx)
    assert F[1:-1] == approx([-kappa*100.0, +kappa*100.0])

Overwriting test_flux.py


In [84]:
!pytest test_flux.py

============================= test session starts ==============================
platform darwin -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0
rootdir: /Users/altuntas/r3sw/notebooks
plugins: hypothesis-6.136.9, anyio-4.10.0
collected 2 items                                                              

test_flux.py ..                                                          [100%]

============================== 2 passed in 0.43s ===============================


### Divergence telescoping


The finite volume telescoping property states that summing the tendencies (`dudt[i]`) over all cells and multiplying by `dx` must equal the net boundary flux (`F[0] - F[-1]`)


#### Exercise 3.1

Write a test named `test_divergence_telescopes` that verifies the telescoping property of the divergence function.


In [85]:
%%writefile test_divergence.py

from pytest import approx
from heat1d import divergence

def test_divergence_telescopes(dx=1.0):
    ...

Overwriting test_divergence.py


In [86]:
!pytest test_divergence.py

============================= test session starts ==============================
platform darwin -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0
rootdir: /Users/altuntas/r3sw/notebooks
plugins: hypothesis-6.136.9, anyio-4.10.0
collected 1 item                                                               

test_divergence.py .                                                     [100%]

============================== 1 passed in 0.18s ===============================


**Answer:**

Below cell implements a test for the telescoping property.

In [87]:
%%writefile test_divergence.py

from pytest import approx
from heat1d import divergence

def test_divergence_telescopes(dx=1.0):
    """Sum of dudt * dx must equal net boundary flux F[0] - F[-1]."""
    F = [2.0, 7.0, -5.0, -3.0]  # N=3 → dudt length=3
    dudt = [0.0, 0.0, 0.0]
    divergence(dudt, F, dx)
    assert sum(dudt) * dx == approx(F[0] - F[-1])

Overwriting test_divergence.py


In [88]:
!pytest test_divergence.py

============================= test session starts ==============================
platform darwin -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0
rootdir: /Users/altuntas/r3sw/notebooks
plugins: hypothesis-6.136.9, anyio-4.10.0
collected 1 item                                                               

test_divergence.py .                                                     [100%]

============================== 1 passed in 0.21s ===============================


# Step & Solve (physical invariants + stability)

Finally, we check several physical invariants and stability constraints end-to-end. The purpose of each test is summarized in the docstrings(`"""..."""`) within each test function.

In [89]:
%%writefile test_step_solve.py

from pytest import approx, raises
from heat1d import Mesh, BC, step, solve

def test_step_moves_spike_toward_neighbors(mesh3, insulated, u_spike):
    """Given insulated BCs and a stable dt, a single step should diffuse the spike:
        - middle cell decreases, neighbors increase."""
    F = mesh3.face_centered_arr()
    dudt = mesh3.cell_centered_arr()
    u = u_spike[:]
    step(u, F, dudt, kappa=0.1, dt=0.1, mesh=mesh3, bc=insulated)
    assert u[1] < 100.0 and u[0] > 0.0 and u[2] > 0.0

def test_conservation_insulated_solve():
    """With qL=qR=0, total discrete heat (sum(u)*dx) is invariant across steps."""
    u0 = [0.0, 100.0, 0.0]
    u  = solve(u0=u0, kappa=0.1, dt=0.1, nt=20, dx=1.0, bc=BC(0.0, 0.0))
    assert sum(u) == approx(sum(u0))

def test_conservation_with_boundary_work():
    """With qL!=qR, total heat changes by dt*(qL - qR) per step."""
    u0 = [10.0, 10.0, 10.0]
    dx, dt, nt = 1.0, 0.05, 4
    bc = BC(qL=2.0, qR=-3.0)  # net in = 5
    u  = solve(u0=u0, kappa=0.1, dt=dt, nt=nt, dx=dx, bc=bc)
    expected = sum(u0)*dx + nt*dt*(bc.qL - bc.qR)
    assert sum(u)*dx == approx(expected)
    # NOTE: bug above on purpose to show failing message; fix to bc.qR in next test.

def test_symmetry_preserved_one_step(mesh3, insulated):
    """A symmetric initial state (a,b,a) under insulated BCs remains symmetric after 1 step."""
    u0 = [0.0, 100.0, 0.0]
    u  = solve(u0, kappa=0.1, dt=0.1, nt=1, dx=1.0, bc=insulated)
    assert u[0] == approx(u[2])

def test_unstable_dt_raises():
    """ Stability guard for dx=1, kappa=0.1. Pick dt=10 to force assert."""
    u0 = [0.0, 100.0, 0.0]
    with raises(AssertionError):
        solve(u0=u0, kappa=0.1, dt=10.0, nt=1, dx=1.0, bc=BC(0.0, 0.0))

def test_uniform_is_fixed_point(mesh5, insulated, u_uniform):
    """Uniform field is a fixed point (steady state) under insulated BCs for any stable dt/kappa."""
    u = solve(u_uniform, kappa=5.0, dt=0.05, nt=10, dx=mesh5.dx, bc=insulated)
    assert u == approx(u_uniform)


def test_equal_flux_bc_trends_toward_linear_profile(mesh5, kappa, linear_bc):
    """ If qL==qR==c (nonzero), steady state has constant interior flux == c and thus a linear
    gradient. This test checks that after many steps the cell differences approach a constant."""
    u0 = [0.0, 0.0, 0.0, 0.0, 0.0]
    # stable dt: r = kappa*dt/dx^2; choose small dt to be safe
    u  = solve(u0, kappa=0.1, dt=0.5, nt=400, dx=mesh5.dx, bc=linear_bc)
    diffs = [u[i]-u[i-1] for i in range(1, len(u))]
    # Differences should be (approximately) equal across cells
    avg = sum(diffs)/len(diffs)
    assert diffs == approx([avg]*len(diffs), rel=1e-3, abs=1e-3)


Overwriting test_step_solve.py


In [90]:
!pytest test_step_solve.py

============================= test session starts ==============================
platform darwin -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0
rootdir: /Users/altuntas/r3sw/notebooks
plugins: hypothesis-6.136.9, anyio-4.10.0
collected 7 items                                                              

test_step_solve.py .......                                               [100%]

============================== 7 passed in 0.25s ===============================


## Summary

We now have a comprehensive suite of unit tests for the 1D heat equation solver.
We can use this test suite to validate any changes or additions to the solver's code.
To re-run all of these tests, one can simply execute the `pytest` command.

To list all available tests, the `pytest --collect-only` command can be used.


In [91]:
!pytest --collect-only

============================= test session starts ==============================
platform darwin -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0
rootdir: /Users/altuntas/r3sw/notebooks
plugins: hypothesis-6.136.9, anyio-4.10.0
collected 14 items                                                             

<Dir notebooks>
  <Module test_div.py>
    <Function test_division>
  <Module test_divergence.py>
    <Function test_divergence_telescopes>
  <Module test_flux.py>
    <Function test_flux_constant_field_yields_zero_interior>
    <Function test_flux_spike_has_opposite_signed_fluxes>
  <Module test_flux_via_params.py>
    <Function test_flux_param[constant\u2192zero]>
    <Function test_flux_param[linear\u2192const-flux]>
  <Module test_simple.py>
    <Function test_flux_simple>
  <Module test_step_solve.py>
    <Function test_step_moves_spike_toward_neighbors>
    <Function test_conservation_insulated_solve>
    <Function test_conservation_with_boundary_work>
    <Function test_symmetry

## Limitations of Unit Testing (and how we’ll push beyond)

Unit tests are necessary but not sufficient:

- **Limited Coverage**: Handpicked inputs may miss edge cases or unexpected behaviors.
- **Repetitive and Tedious**: Writing unit tests can be repetitive and tedious, especially for functions with many parameters or complex logic.
- **Overfitting**: Tests can become too specific, making them brittle and hard to maintain. If the implementation changes, the tests may need to be rewritten, even if the overall behavior remains correct.

## Looking Ahead:

In Chapter 4, we’ll encode properties (conservation, symmetry, maximum-principle intuition, stability ranges) and let a generator explore many inputs automatically